In [2]:
from abc import ABC, abstractmethod
from typing import List, Any
import copy

class Command(ABC):
    """命令抽象基类"""
    
    @abstractmethod
    def execute(self) -> Any:
        """执行命令"""
        pass
    
    @abstractmethod
    def undo(self) -> Any:
        """撤销命令"""
        pass
    
    @abstractmethod
    def get_description(self) -> str:
        """获取命令描述"""
        pass

class Receiver:
    """接收者 - 实际执行操作的对象"""
    
    def __init__(self):
        self.shapes = []  # 存储几何体
        self.current_id = 0
    
    def add_shape(self, shape_data):
        """添加几何体"""
        shape_id = self.current_id
        self.shapes.append({
            'id': shape_id,
            'data': shape_data,
            'active': True
        })
        self.current_id += 1
        print(f"✅ 添加几何体 ID: {shape_id}")
        return shape_id
    
    def remove_shape(self, shape_id):
        """删除几何体"""
        for shape in self.shapes:
            if shape['id'] == shape_id and shape['active']:
                shape['active'] = False
                print(f"🗑️ 删除几何体 ID: {shape_id}")
                return True
        return False
    
    def restore_shape(self, shape_id):
        """恢复几何体"""
        for shape in self.shapes:
            if shape['id'] == shape_id and not shape['active']:
                shape['active'] = True
                print(f"🔄 恢复几何体 ID: {shape_id}")
                return True
        return False
    
    def get_active_shapes(self):
        """获取当前活跃的几何体"""
        return [s for s in self.shapes if s['active']]

class Invoker:
    """调用者 - 管理命令历史"""
    
    def __init__(self):
        self.history: List[Command] = []
        self.current_index = -1
    
    def execute_command(self, command: Command):
        """执行命令并记录到历史"""
        # 如果当前不在历史末尾，删除后续历史
        if self.current_index < len(self.history) - 1:
            self.history = self.history[:self.current_index + 1]
        
        # 执行命令
        result = command.execute()
        
        # 添加到历史
        self.history.append(command)
        self.current_index += 1
        
        print(f"📝 执行命令: {command.get_description()}")
        return result
    
    def undo(self):
        """撤销操作"""
        if self.can_undo():
            command = self.history[self.current_index]
            result = command.undo()
            self.current_index -= 1
            print(f"↩️ 撤销命令: {command.get_description()}")
            return result
        else:
            print("❌ 无可撤销的操作")
            return None
    
    def redo(self):
        """重做操作"""
        if self.can_redo():
            self.current_index += 1
            command = self.history[self.current_index]
            result = command.execute()
            print(f"↪️ 重做命令: {command.get_description()}")
            return result
        else:
            print("❌ 无可重做的操作")
            return None
    
    def can_undo(self) -> bool:
        """是否可以撤销"""
        return self.current_index >= 0
    
    def can_redo(self) -> bool:
        """是否可以重做"""
        return self.current_index < len(self.history) - 1
    
    def get_history_summary(self):
        """获取历史记录摘要"""
        print(f"📚 命令历史 (当前位置: {self.current_index + 1}/{len(self.history)}):")
        for i, cmd in enumerate(self.history):
            marker = "➤" if i == self.current_index else " "
            print(f"  {marker} {i}: {cmd.get_description()}")

In [3]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere
from OCC.Core.gp import gp_Pnt

class CreateBoxCommand(Command):
    """创建立方体命令"""
    
    def __init__(self, receiver: Receiver, width: float, height: float, depth: float):
        self.receiver = receiver
        self.width = width
        self.height = height
        self.depth = depth
        self.shape_id = None
    
    def execute(self):
        """执行：创建立方体"""
        box_shape = BRepPrimAPI_MakeBox(self.width, self.height, self.depth).Shape()
        self.shape_id = self.receiver.add_shape({
            'type': 'box',
            'width': self.width,
            'height': self.height,
            'depth': self.depth,
            'shape': box_shape
        })
        return self.shape_id
    
    def undo(self):
        """撤销：删除立方体"""
        if self.shape_id is not None:
            return self.receiver.remove_shape(self.shape_id)
        return False
    
    def get_description(self) -> str:
        return f"创建立方体 ({self.width}×{self.height}×{self.depth})"

class CreateSphereCommand(Command):
    """创建球体命令"""
    
    def __init__(self, receiver: Receiver, center: gp_Pnt, radius: float):
        self.receiver = receiver
        self.center = center
        self.radius = radius
        self.shape_id = None
    
    def execute(self):
        """执行：创建球体"""
        sphere_shape = BRepPrimAPI_MakeSphere(self.center, self.radius).Shape()
        self.shape_id = self.receiver.add_shape({
            'type': 'sphere',
            'center': self.center,
            'radius': self.radius,
            'shape': sphere_shape
        })
        return self.shape_id
    
    def undo(self):
        """撤销：删除球体"""
        if self.shape_id is not None:
            return self.receiver.remove_shape(self.shape_id)
        return False
    
    def get_description(self) -> str:
        return f"创建球体 (半径: {self.radius})"

class DeleteShapeCommand(Command):
    """删除几何体命令"""
    
    def __init__(self, receiver: Receiver, shape_id: int):
        self.receiver = receiver
        self.shape_id = shape_id
    
    def execute(self):
        """执行：删除几何体"""
        return self.receiver.remove_shape(self.shape_id)
    
    def undo(self):
        """撤销：恢复几何体"""
        return self.receiver.restore_shape(self.shape_id)
    
    def get_description(self) -> str:
        return f"删除几何体 (ID: {self.shape_id})"

In [4]:
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

"""命令模式演示"""

print("🎯 命令模式 Undo/Redo 演示")
print("=" * 50)

# 创建接收者和调用者
receiver = Receiver()
invoker = Invoker()

# 初始化渲染器
renderer = JupyterRenderer()

# 创建几个命令
cmd1 = CreateBoxCommand(receiver, 10, 20, 30)
cmd2 = CreateSphereCommand(receiver, gp_Pnt(0, 0, 0), 15)
cmd3 = CreateBoxCommand(receiver, 5, 5, 5)

# 执行命令
print("\n📝 执行命令序列:")
invoker.execute_command(cmd1)
invoker.execute_command(cmd2)
invoker.execute_command(cmd3)

# 显示当前活跃几何体
active_shapes = receiver.get_active_shapes()
for shape in active_shapes:
    renderer.DisplayShape(shape['data']['shape'])
renderer.Display()

print(f"\n当前活跃几何体数量: {len(receiver.get_active_shapes())}")





🎯 命令模式 Undo/Redo 演示

📝 执行命令序列:
✅ 添加几何体 ID: 0
📝 执行命令: 创建立方体 (10×20×30)
✅ 添加几何体 ID: 1
📝 执行命令: 创建球体 (半径: 15)
✅ 添加几何体 ID: 2
📝 执行命令: 创建立方体 (5×5×5)



当前活跃几何体数量: 3


In [5]:
# 撤销操作
print("\n↩️ 撤销操作:")
invoker.undo()  # 撤销创建小立方体
invoker.undo()  # 撤销创建球体


 # 显示当前活跃几何体
renderer.EraseAll()
active_shapes = receiver.get_active_shapes()
for shape in active_shapes:
    renderer.DisplayShape(shape['data']['shape'])
renderer.Display()

print(f"撤销后活跃几何体数量: {len(receiver.get_active_shapes())}")




↩️ 撤销操作:
🗑️ 删除几何体 ID: 2
↩️ 撤销命令: 创建立方体 (5×5×5)
🗑️ 删除几何体 ID: 1
↩️ 撤销命令: 创建球体 (半径: 15)


撤销后活跃几何体数量: 1


In [6]:
# 重做操作
print("\n↪️ 重做操作:")
invoker.redo()  # 重做创建球体
 # 显示当前活跃几何体
renderer.EraseAll()
active_shapes = receiver.get_active_shapes()
for shape in active_shapes:
    renderer.DisplayShape(shape['data']['shape'])
renderer.Display()
print(f"重做后活跃几何体数量: {len(receiver.get_active_shapes())}")




↪️ 重做操作:
✅ 添加几何体 ID: 3
↪️ 重做命令: 创建球体 (半径: 15)


重做后活跃几何体数量: 2


In [7]:
# 显示历史记录
print("\n")
invoker.get_history_summary()

# 新命令会清除后续历史
print("\n📝 执行新命令（将清除后续历史）:")
cmd4 = CreateSphereCommand(receiver, gp_Pnt(10, 10, 10), 8)
invoker.execute_command(cmd4)

print("\n")
invoker.get_history_summary()



📚 命令历史 (当前位置: 2/3):
    0: 创建立方体 (10×20×30)
  ➤ 1: 创建球体 (半径: 15)
    2: 创建立方体 (5×5×5)

📝 执行新命令（将清除后续历史）:
✅ 添加几何体 ID: 4
📝 执行命令: 创建球体 (半径: 8)


📚 命令历史 (当前位置: 3/3):
    0: 创建立方体 (10×20×30)
    1: 创建球体 (半径: 15)
  ➤ 2: 创建球体 (半径: 8)
